In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np, pandas as pd

In [0]:
df = pd.read_csv('/content/drive/My Drive/list_of_sentences', header = None)

In [0]:
list_of_sent = df.astype(str).values.tolist()

In [0]:
GLOVE_DIR = '/content/drive/My Drive/glove.6B.50d.txt'

In [0]:
def loadGloveModel(GLOVE_DIR):
    print ("Loading Glove Model")
    with open(GLOVE_DIR, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

In [16]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import pandas as pd

def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    #print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')
    return round((1-cosine)*100,2)

In [18]:
model = loadGloveModel(GLOVE_DIR)

Loading Glove Model
Done. 400000  words loaded!


In [0]:
s , t = "Football is played in Brazil", "Cricket is played in India"
u , v =  "Traveling is good for health", "People love traveling in winter"
w,  x = "nokia", "samsung"

In [20]:
print(cosine_distance_wordembedding_method(s, t))
print(cosine_distance_wordembedding_method(w, x))

79.48
70.86


In [0]:
def output(list_of_words):

  op = []
  #sentences = [list_of_words]
  for i in range(len(list_of_words)):
    for j in range(1, i):
      if cosine_distance_wordembedding_method(list_of_words[i][0], list_of_words[j][0]) > 70.0:
        op.extend([[list_of_words[i], list_of_words[j]]])
  
  return op

In [29]:
output(list_of_sent)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[[['good afternoon'], ['the weather is awesome today']],
 [['are you doing good ?'], ['the weather is awesome today']],
 [['are you doing good ?'], ['good afternoon']],
 [['nokia'], ['samsung']],
 [['cricket is a fun game'], ['good afternoon']],
 [['cricket is a fun game'], ['baseball is played in the USA']],
 [['cricket is a fun game'], ['are you doing good ?']],
 [['the climate change is a problem'], ['the weather is awesome today']],
 [['the climate change is a problem'], ['are you doing good ?']]]